In [ ]:
!pip install python-louvain

-- Importation des bibliothèques et packages importants

In [38]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn import metrics
from sklearn.metrics import silhouette_score, adjusted_rand_score
import community
from community import community_louvain
from networkx.algorithms.community import asyn_lpa_communities
import matplotlib.pyplot as plt
from random import randint
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
import numpy as np
from sklearn.preprocessing import StandardScaler

%matplotlib inline

-- Importation et Préparation des données

In [39]:
facebook = pd.read_csv("out.facebook-wosn-links", sep=' ', names=['source', 'target', 'status', 'timestamp'],nrows=1000)
facebook

In [40]:
# Supprimer les lignes avec des valeurs non numériques
facebook = facebook[pd.to_numeric(facebook['source'], errors='coerce').notnull()]
facebook = facebook[pd.to_numeric(facebook['target'], errors='coerce').notnull()]

In [41]:
G = nx.from_pandas_edgelist(facebook, "source", "target")

In [42]:
plt.figure(figsize=(10, 6))
nx.draw(G, with_labels=False, node_size=100, font_size=8)
plt.title("Graphe des amitiés sur Facebook (échantillon)")


In [43]:
print (f"Nombre des noeuds : {G.number_of_nodes()}")
print (f"Nombre des aretes : {G.number_of_edges()}")

In [44]:
features = facebook[['source', 'target' , 'status']]
scaler = StandardScaler()
feature_matrix = scaler.fit_transform(features)
print("",feature_matrix)

In [45]:
# Créer une matrice d'adjacence binaire pour la relation "friend" entre les paires de nœuds
adjacency_matrix = np.asarray(nx.to_numpy_array(G, dtype=int))

# **Utilisation des Algorithmes de Clustring pour détecter les communautées :**

-- Application de l'algorithme  **K-means** :

In [46]:
# Liste des nombres de clusters à tester
num_clusters_list = range(2, 11)  # Vous pouvez ajuster cette plage selon vos besoins

# Stocker les scores de silhouette pour chaque nombre de clusters
silhouette_scores = []

# Essayer différents nombres de clusters
for num_clusters in num_clusters_list:
    gmm = GaussianMixture(n_components=num_clusters, random_state=42)
    cluster_labels = gmm.fit_predict(adjacency_matrix)
    silhouette_avg = silhouette_score(adjacency_matrix, cluster_labels)
    silhouette_scores.append(silhouette_avg)

# Trouver l'indice du maximum dans la liste des scores de silhouette
optimal_num_clusters = num_clusters_list[np.argmax(silhouette_scores)]

# Tracer la courbe des scores de silhouette en fonction du nombre de clusters
plt.plot(num_clusters_list, silhouette_scores, marker='o')
plt.xlabel('Nombre de Clusters')
plt.ylabel('Score de Silhouette')
plt.title('Méthode de l\'Indice de Silhouette pour Trouver le Nombre Optimal de Clusters')
plt.show()

# Afficher le nombre optimal de clusters
print(f'Le nombre optimal de clusters est : {optimal_num_clusters}')

In [47]:
kmeans = KMeans(n_clusters=optimal_num_clusters)
node_labels = kmeans.fit_predict(adjacency_matrix)
kmeans_labels = kmeans.labels_

# Visualisation des resultats

unique_labels = np.unique(node_labels)
color_map = plt.cm.get_cmap('viridis', len(unique_labels))

plt.figure(figsize=(10, 8))
nx.draw(G, with_labels=False, font_weight='bold', node_size=100, node_color=node_labels, cmap=color_map)
plt.title('Resultats de Clustring K-Means')
plt.show()


-- Application de l'algorithme  **DBSCAN** :

In [48]:
dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan_labels = dbscan.fit_predict(adjacency_matrix)

# Visualisation des resultats
plt.figure(figsize=(10, 8))
nx.draw(G, with_labels=False, font_weight='bold', node_size=50, node_color=dbscan_labels, cmap='viridis')
plt.title('Resultats de Clustring avec DBSCAN')
plt.show()

In [49]:
from sklearn.mixture import GaussianMixture

# Essayez différentes valeurs pour le nombre de composants
components_range = range(1, 11)
bic_scores = []

for num_components in components_range:
    gmm = GaussianMixture(n_components=num_components, random_state=42)
    gmm.fit(adjacency_matrix)
    bic_scores.append(gmm.bic(adjacency_matrix))

# Trouvez le nombre optimal de composants qui minimise le score BIC
optimal_components = components_range[np.argmax(bic_scores)]
optimal_components

-- Application de l'algorithme **GMM** :

In [50]:

# Appliquer le modèle de mélange gaussien (GMM) pour effectuer le clustering
num_clusters = 4
gmm = GaussianMixture(n_components=num_clusters, random_state=42)
labels = gmm.fit_predict(adjacency_matrix)

# Assign cluster labels to nodes
node_colors = {node: label for node, label in zip(G.nodes, labels)}

# Plot the graph with node colors based on the GMM clusters
pos = nx.spring_layout(G)
plt.figure(figsize=(10, 8))
nx.draw(G, with_labels=False, font_weight='bold', node_size=50, node_color=node_labels, cmap='viridis')
plt.title("Graph with Node Colors Based on GMM Clusters")
plt.show()

# **Utilisation des méthodes classiques pour détecter les communautés :**

-- Application de l'algorithme  **Label Propagation** :

In [51]:
lb_communities = list(asyn_lpa_communities(G))
lb_part = [i for i, comm in enumerate(lb_communities) for _ in comm]

# Visualisation des résultats

plt.figure(figsize=(8, 6))
plt.title('Label Propagation Community Detection')
node_colors = [i for i, comm in enumerate(lb_communities) for _ in comm]
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=False, node_color=node_colors, cmap=plt.cm.tab10, node_size=300, edge_color='gray')
plt.show()

-- Application de l'algorithme **Louvain**

In [52]:
louvain_partition = community_louvain.best_partition(G)
louvain_part= [i for i, comm in enumerate(louvain_partition) for _ in comm]

# Visualisation des résultats

pos = nx.spring_layout(G)
colors = [louvain_partition[node] for node in G.nodes]

plt.figure(figsize=(10, 8))
nx.draw(G, pos, with_labels=False, node_color=node_colors, cmap=plt.cm.tab10, node_size=300, edge_color='gray')
plt.title("Louvain Community Detection")
plt.show()

In [53]:
def girvan_newman(G):
    communities = list(nx.connected_components(G))
    while len(communities) == 1:
        edge_betweenness = nx.edge_betweenness_centrality(G)
        max_betweenness = max(edge_betweenness.values())
        edges_to_remove = [edge for edge, centrality in edge_betweenness.items() if centrality == max_betweenness]
        G.remove_edges_from(edges_to_remove)
        communities = list(nx.connected_components(G))
    return communities

Girvan_Newman_communities = girvan_newman(G)
gn_part= [i for i, comm in enumerate(Girvan_Newman_communities) for _ in comm]

# Visualization
pos = nx.spring_layout(G)
colors = [i for node in G.nodes() for i, comm in enumerate(Girvan_Newman_communities) if node in comm]

plt.figure(figsize=(10, 8))
nx.draw(G, pos, with_labels=False, font_weight='bold', node_color=colors, cmap=plt.cm.tab10, node_size=300, edge_color='gray')
plt.title("Girvan-Newman Community Detection")
plt.show()


# **Evaluation des algorithmes **

**Silhouette Score**

In [54]:

silhouette_kmeans = silhouette_score(adjacency_matrix, node_labels)

silhouette_dbscan = silhouette_score(adjacency_matrix,dbscan_labels)

silhouette_gmm = silhouette_score(adjacency_matrix, labels)

silhouette_lb = silhouette_score(feature_matrix[0:823],  lb_part )

silhouette_louvain = silhouette_score(feature_matrix, louvain_part[:len(feature_matrix)])

silhouette_GN = silhouette_score(feature_matrix[0:823], gn_part)

silhouette_scores ={ 'K-Means': [silhouette_kmeans] , 'DBSCAN' : [silhouette_dbscan] , 'GMM' : [silhouette_gmm] , 'Label Propagation': [silhouette_lb] , 'Louvain' : [silhouette_louvain] , 'Girvan-Newman': [silhouette_GN]}
silhouette_scores_df = pd.DataFrame(silhouette_scores, index=['Silhouette Score'])
silhouette_scores_df

**Adjusted Rand Score**( comparaison des algorithmes de clustring avec l'algorithme classique Girvan-Newman  )

In [55]:
ari_gn_kmeans = adjusted_rand_score(gn_part, kmeans_labels[0:823])

ari_gn_dbscan = adjusted_rand_score(gn_part, dbscan_labels)

ari_gn_gmm = adjusted_rand_score(gn_part, labels[0:823])

ari_scores ={'K-means':[ari_gn_kmeans],'DBSCAN':[ari_gn_dbscan] ,'GMM':[ari_gn_gmm]}
ari_scores_df= pd.DataFrame(ari_scores,index=['Adjusted Rand score'])
ari_scores_df

# **Les mesures de centralités :**

Centralité de Degré :

In [56]:
degree_centrality = nx.degree_centrality(G)
(sorted(G.degree, key=lambda item: item[1], reverse=True))[:10]

In [57]:
plt.figure(figsize=(15, 8))
plt.hist(degree_centrality.values(), bins=25)
plt.xticks(ticks=[0, 0.025, 0.05, 0.1, 0.15, 0.2])  # set the x axis ticks
plt.title("Degree Centrality Histogram ", fontdict={"size": 35}, loc="center")
plt.xlabel("Degree Centrality", fontdict={"size": 20})
plt.ylabel("Counts", fontdict={"size": 20})

###  les utilisateurs avec le plus haut degré de centralité de par la taille de leurs nœuds :

In [58]:
node_size = [
    v * 1000 for v in degree_centrality.values()
]  # set up nodes size for a nice graph representation
plt.figure(figsize=(15, 8))
nx.draw_networkx(G, pos=pos, node_size=node_size, with_labels=False, width=0.15)
plt.axis("off")

Centralité de vecteur propre :

In [59]:
eigenvector_centrality = nx.eigenvector_centrality(G)
(sorted(eigenvector_centrality.items(), key=lambda item: item[1], reverse=True))[:10]

Centralité de Proximité :

In [60]:
closeness_centrality = nx.closeness_centrality(G)
(sorted(closeness_centrality.items(), key=lambda item: item[1], reverse=True))[:10]

###  la répartition des centralités de proximité :

In [61]:
plt.figure(figsize=(15, 8))
plt.hist(closeness_centrality.values(), bins=60)
plt.title("Closeness Centrality Histogram ", fontdict={"size": 35}, loc="center")
plt.xlabel("Closeness Centrality", fontdict={"size": 20})
plt.ylabel("Counts", fontdict={"size": 20})

### certaines communautés situées plus loin, dont les nœuds auraient les centralités de proximité minimales, comme on le voit ci-dessous :

In [62]:
node_size = [
    v * 50 for v in closeness_centrality.values()
]  # set up nodes size for a nice graph representation
plt.figure(figsize=(15, 8))
nx.draw_networkx(G, pos=pos, node_size=node_size, with_labels=False, width=0.15)
plt.axis("off")

Centralité d'intermédiarité

In [63]:
betweenness_centrality = nx.betweenness_centrality(G)
(sorted(betweenness_centrality.items(), key=lambda item: item[1], reverse=True))[:10]

In [64]:
plt.figure(figsize=(15, 8))
plt.hist(betweenness_centrality.values(), bins=100)
plt.xticks(ticks=[0, 0.02, 0.1, 0.2, 0.3, 0.4, 0.5])  # set the x axis ticks
plt.title("Betweenness Centrality Histogram ", fontdict={"size": 35}, loc="center")
plt.xlabel("Betweenness Centrality", fontdict={"size": 20})
plt.ylabel("Counts", fontdict={"size": 20})

### image sur les nœuds avec les centralités intermédiaires les plus élevées et où ils se trouvent dans le réseau. Il est clair qu’ils constituent des ponts d’une communauté à une autre :

In [65]:
node_size = [
    v * 1200 for v in betweenness_centrality.values()
]  # set up nodes size for a nice graph representation
plt.figure(figsize=(15, 8))
nx.draw_networkx(G, pos=pos, node_size=node_size, with_labels=False, width=0.15)
plt.axis("off")